In [1]:
import trimesh
import pyrender
import numpy as np
import os

# --- Configuration ---
# IMPORTANT: Make sure this path is correct for your WSL environment.
# This assumes your 'rtsc-1.6' folder is directly under your home directory.
dataset_root = r"/home/jeans/progressive_img2sketch/rtsc-1.6/LOD_data_50"

# --- Load a sample mesh for visualization ---
# Let's visualize lod1 from scene 0 as an example.
# You can change '0' to another scene ID and 'lod1' to 'lod2', 'lod3', 'lod4'.
scene_id = "0"
lod_name = "lod1"
mesh_path = os.path.join(dataset_root, scene_id, f"{lod_name}.obj")

print(f"Attempting to load mesh from: {mesh_path}")

try:
    # Trimesh can handle OBJ, MTL, and textures automatically if paths are correct
    mesh = trimesh.load(mesh_path)

    # Handle Trimesh Scenes (if the OBJ loads as a scene with multiple parts)
    if isinstance(mesh, trimesh.scene.Scene):
        combined_meshes = []
        for geo_name, geo_obj in mesh.geometry.items():
            if isinstance(geo_obj, trimesh.Trimesh):
                combined_meshes.append(geo_obj)
            elif isinstance(geo_obj, trimesh.scene.Scene): # Handle potentially nested scenes
                 for sub_geo_name, sub_geo_obj in geo_obj.geometry.items():
                     if isinstance(sub_geo_obj, trimesh.Trimesh):
                         combined_meshes.append(sub_geo_obj)
        
        if combined_meshes:
            mesh = trimesh.util.concatenate(tuple(combined_meshes))
            print(f"Concatenated {len(combined_meshes)} sub-meshes into a single mesh.")
        else:
            print(f"Warning: {mesh_path} is a Scene but contains no renderable Trimesh objects.")
            mesh = None # Set mesh to None to prevent further processing
    
    # Check if the loaded mesh is valid before proceeding
    if mesh is None or mesh.is_empty or mesh.vertices.shape[0] == 0 or mesh.faces.shape[0] == 0:
        print(f"Error: Loaded mesh from {mesh_path} is empty or invalid. Cannot visualize.")
    else:
        print(f"Successfully loaded mesh with {len(mesh.vertices)} vertices and {len(mesh.faces)} faces.")
        
        # --- Create Pyrender Scene ---
        # Using a slightly darker background for better visibility of light models
        scene = pyrender.Scene(bg_color=[0.1, 0.1, 0.1, 1.0])

        # Add the mesh to the scene.
        # pyrender.Mesh.from_trimesh will automatically use textures if trimesh loaded them.
        # Use smooth=True for a nicer visual appearance in the viewer.
        mesh_node = pyrender.Mesh.from_trimesh(mesh, smooth=True)
        scene.add(mesh_node)

        # --- Add Simple Lighting for Viewer ---
        # The Viewer often adds default lighting, but explicit lights ensure good visibility.
        # A simple directional light from the front-top-right
        light = pyrender.DirectionalLight(color=np.ones(3), intensity=2.0) # White light, moderate intensity
        light_pose = np.array([
            [1.0, 0.0, 0.0, 5.0],
            [0.0, 1.0, 0.0, 5.0],
            [0.0, 0.0, 1.0, 5.0],
            [0.0, 0.0, 0.0, 1.0],
        ])
        scene.add(light, pose=light_pose)

        # --- Create and Display Interactive Viewer ---
        print("\nOpening Pyrender Viewer. Interact with the model (rotate, pan, zoom).")
        print("Close the viewer window to proceed with the notebook cell execution.")

        # `use_raymond_lighting=True` provides a nice default lighting setup for the viewer.
        # `run_in_thread=True` can be useful in notebooks to prevent blocking,
        # but sometimes can cause issues depending on your environment.
        # If the viewer window doesn't appear or crashes, try removing `run_in_thread=True`.
        v = pyrender.Viewer(scene, 
                            viewport_width=800, 
                            viewport_height=600,
                            use_raymond_lighting=True,
                            # run_in_thread=True 
                           )
        
        print("Viewer closed.")

except Exception as e:
    print(f"An error occurred during mesh loading or visualization: {e}")

Attempting to load mesh from: /home/jeans/progressive_img2sketch/rtsc-1.6/LOD_data_50/0/lod1.obj
Successfully loaded mesh with 360 vertices and 192 faces.

Opening Pyrender Viewer. Interact with the model (rotate, pan, zoom).
Close the viewer window to proceed with the notebook cell execution.


: 